In [3]:
from folktables import ACSDataSource
import folktables
import pandas as pd
import numpy as np
import random
import pickle
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm
from sklearn.linear_model import LinearRegression, Ridge, Lasso

import matplotlib.pyplot as plt
from folktables.load_acs import state_list

# Baseline LS and Ridge

## on one hot

In [4]:
df_oh_all = pd.read_pickle("./data_frames/dense_acs_mm_oh.pkl")

In [5]:
df_oh_all.head()

,WKHP,AGEP,PINCP,OCCP_10,OCCP_20,OCCP_40,OCCP_51,OCCP_52,OCCP_60,OCCP_101,...,MAR_5,RAC1P_1,RAC1P_2,RAC1P_3,RAC1P_4,RAC1P_5,RAC1P_6,RAC1P_7,RAC1P_8,RAC1P_9
0,0.295918,0.025316,0.039472,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,0.397959,0.037975,0.030967,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,0.173469,0.025316,0.030467,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,0.051020,0.215190,0.053479,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0.091837,0.025316,0.009455,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


In [6]:
from sklearn.model_selection import train_test_split
rand_seed = 21 
X_train, X_test, y_train, y_test = train_test_split(df_oh_all.drop('PINCP', axis=1), 
                                                    df_oh_all['PINCP'], test_size = 0.2,
                                                    random_state = rand_seed)

In [34]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1255350, 897), (1255350,), (313838, 897), (313838,))

In [ ]:
lr = LinearRegression() #sklearn
lr.fit(X_train, y_train)

In [12]:
lr.score(X_train, y_train), lr.score(X_test, y_test)

(0.5085280100557967, 0.5089598328450473)

In [13]:
ridge = Ridge()
ridge.fit(X_train, y_train)

/home/jupyter-kriach/.conda/envs/biasbounties/lib/python3.10/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Ridge()

In [39]:
np.random.rand(y_train.shape[0]) # uniform random predictor

array([0.12039289, 0.63934487, 0.14759233, ..., 0.95642588, 0.18653082,
       0.93438063])

In [40]:
r2_score(y_train, np.random.rand(y_train.shape[0]))

-3.4789383966767655

In [44]:
r2_score(y_test, y_test)

1.0

In [14]:
ridge.score(X_train, y_train), ridge.score(X_test, y_test)

(0.5085694610333208, 0.5089990692196149)

In [16]:
np.allclose(ridge.coef_, lr.coef_, rtol=0.01)

False

In [17]:
ridge.coef_[:5], lr.coef_[:5]

(array([0.33726913, 0.16453486, 0.17282151, 0.08041582, 0.07610074]),
 array([3.37420660e-01, 1.64678596e-01, 6.64869973e+09, 6.64869973e+09,
        6.64869973e+09]))

In [29]:
np.all(ridge.coef_ < 0.4), np.all(lr.coef_ < 1) # ridge always learns small params; obviously regularization

(True, False)

In [30]:
np.all(ridge.coef_>-0.4), np.all(lr.coef_ > -1) # ridge coeff all between [-0.4,0.4], [-1, 1]

(True, False)

In [ ]:
# real thought experiment
- random values for y now, X still the same

In [47]:
lr_the = LinearRegression()
y_rand_train = np.random.rand(X_train.shape[0])
lr_the.fit(X_train, y_rand_train)

LinearRegression()

In [48]:
y_rand_test = np.random.rand(X_test.shape[0])
lr_the.score(X_train, y_rand_train), lr_the.score(X_test, y_rand_test)

(0.0006347467266070161, -0.0007181169604526527)

In [49]:
lr_the.predict(X_test)

array([0.50263214, 0.50217438, 0.50003815, ..., 0.50563812, 0.50763702,
       0.49964142])

## Ridge and LS on not one hot

In [55]:
df_notoh = pd.read_pickle("./data_frames/dense_acs_mm_notoh.pkl")

In [24]:
cat_cols_sig = ['OCCP', 'SCHL', 'ST', 'JWTRNS', 'DRAT', 'COW', 'SEX', \
       'RELSHIPP', 'POBP', 'ENG', 'MAR', 'RAC1P'] # significant features from the earlier analysis
numeric_cols = ['WKHP', 'AGEP', 'PINCP']
print(len(cat_cols_sig), len(numeric_cols))

12 3


In [25]:
from utils import ordinal_encoder
df_notoh = ordinal_encoder(df_notoh, cat_cols_sig)

In [26]:
from sklearn.model_selection import train_test_split
rand_seed = 21
X_train_noh, X_test_noh, y_train_noh, y_test_noh = train_test_split(df_notoh.drop('PINCP', axis=1), 
                                                    df_notoh['PINCP'], test_size = 0.2,
                                                    random_state = rand_seed)

In [28]:
X_train_noh.shape, y_train_noh.shape, X_test_noh.shape, y_test_noh.shape

((1255350, 14), (1255350,), (313838, 14), (313838,))

In [12]:
lr_notoh = LinearRegression()
lr_notoh.fit(X_train, y_train)

LinearRegression()

In [13]:
lr_notoh.score(X_train, y_train), lr_notoh.score(X_test, y_test)

(0.39213866332368696, 0.3933595075579419)

In [14]:
ridge_notoh = Ridge()
ridge_notoh.fit(X_train, y_train)

Ridge()

In [15]:
ridge_notoh.score(X_train, y_train), ridge_notoh.score(X_test, y_test)

(0.3921386631275038, 0.39335948968321377)

In [18]:
np.allclose(ridge_notoh.coef_, lr_notoh.coef_)

False

In [22]:
np.all(ridge_notoh.coef_ < 1), np.all(lr_notoh.coef_ < 1) # ridge always learns small params; obviously regularization

(True, True)

In [23]:
np.all(ridge_notoh.coef_>-1), np.all(lr_notoh.coef_ > -1) # ridge coeff all between [-0.4,0.4], [-1, 1]

(True, True)

# non linear below

# kNN regressor on ordinal encoded

In [28]:
from sklearn.neighbors import KNeighborsRegressor
neigh10 = KNeighborsRegressor(n_neighbors = 10)

In [29]:
%%time
neigh10.fit(X_train, y_train)

CPU times: user 3.77 s, sys: 38.2 ms, total: 3.81 s
Wall time: 3.79 s


KNeighborsRegressor(n_neighbors=10)

In [30]:
%%time
neigh10.score(X_train, y_train), neigh10.score(X_test, y_test)

CPU times: user 3min 44s, sys: 176 ms, total: 3min 44s
Wall time: 3min 44s


(0.49791175065745363, 0.38774123935632665)

In [6]:
import numpy as np
import faiss

class FaissKNeighbors_reg:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        self.y = y

    def predict(self, X):
        distances, indices = self.index.search(X.astype(np.float32), k=self.k)
        labels = self.y[indices]
        # predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return labels.mean(axis = 1)

class FaissKNeighbors_reg_gpu:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.y = y
        self.res = faiss.StandardGpuResources()
        index_flat = faiss.IndexFlatL2(X.shape[1])
        self.gpu_index_flat = faiss.index_cpu_to_gpu(self.res, 0, index_flat)
        self.gpu_index_flat.add(X.astype(np.float32))

    def predict(self, X):
        distances, indices = self.gpu_index_flat.search(X.astype(np.float32), k=self.k)
        labels = self.y[indices]
        # predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return labels.mean(axis = 1)

In [58]:
neigh_faiss10 = FaissKNeighbors_reg(k=10)
neigh_faiss10.fit(X_train, y_train.to_numpy())

In [59]:
%%time # cpu search
y_test_pred10 = neigh_faiss10.predict(X_test)

CPU times: user 15min 59s, sys: 46.7 s, total: 16min 46s
Wall time: 1min 3s


In [60]:
r2_score(y_test.to_numpy(), y_test_pred10)

0.3874510991246196

In [62]:
neigh_faiss10gpu = FaissKNeighbors_reg_gpu(k=10)
neigh_faiss10gpu.fit(X_train, y_train.to_numpy())

In [64]:
y_test_pred10gpu = neigh_faiss10gpu.predict(X_test)

In [65]:
r2_score(y_test.to_numpy(), y_test_pred10gpu)

0.3874757098268281

In [33]:
print(index.ntotal)

1255350


In [44]:
print(y_train.iloc[0], y_train.iloc[346723], y_train.iloc[1010530],y_train.iloc[1169275])
(y_train.iloc[0]+y_train.iloc[346723]+y_train.iloc[1010530]+y_train.iloc[1169275])/4


0.05948271549352144 0.004452448846865777 0.29461203662014107 0.24968732803041677


0.15205863224773625

## knn on one hot encoded

In [5]:
df_oh_all = pd.read_pickle("./data_frames/dense_acs_mm_oh.pkl")

In [6]:
from sklearn.model_selection import train_test_split
rand_seed = 21 
X_train, X_test, y_train, y_test = train_test_split(df_oh_all.drop('PINCP', axis=1), 
                                                    df_oh_all['PINCP'], test_size = 0.2,
                                                    random_state = rand_seed)

In [7]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1255350, 897), (1255350,), (313838, 897), (313838,))

In [28]:
neigh_faiss10gpu = FaissKNeighbors_reg_gpu(k=100)
neigh_faiss10gpu.fit(X_train, y_train.to_numpy())

In [29]:
import time
t1 = time.time()
y_test_pred10gpu_oh = neigh_faiss10gpu.predict(X_test)
t2 = time.time()
print(t2-t1)

55.93799138069153


In [30]:
r2_score(y_test, y_test_pred10gpu_oh)

0.46308605613952636

In [8]:
# on ordinal encoded
y_train_np = y_train.to_numpy()
best_k = -100
max_r2_score = -100
for nn in range(1, 100):
    neigh_faissgpu = FaissKNeighbors_reg_gpu(k = nn)
    neigh_faissgpu.fit(X_train, y_train_np)
    y_test_predgpu_oh = neigh_faissgpu.predict(X_test)
    r2_score_now = r2_score(y_test, y_test_predgpu_oh)
    print(f'''num neighbors {nn}, r2 score {r2_score_now}''')
    if r2_score_now > max_r2_score:
        max_r2_score = r2_score_now
        best_k = nn

num neighbors 1, r2 score -0.025556826968981206
num neighbors 2, r2 score 0.2124044295805374
num neighbors 3, r2 score 0.2879947689375989
num neighbors 4, r2 score 0.3256705994881345
num neighbors 5, r2 score 0.3482427102859478
num neighbors 6, r2 score 0.36163376960257987
num neighbors 7, r2 score 0.3711437347858296
num neighbors 8, r2 score 0.3784406259074764
num neighbors 9, r2 score 0.3834860242399991
num neighbors 10, r2 score 0.3874757098268281
num neighbors 11, r2 score 0.39051904732963616
num neighbors 12, r2 score 0.3929191699035599
num neighbors 25, r2 score 0.4003719853795501
num neighbors 26, r2 score 0.4002578964919725
num neighbors 27, r2 score 0.4001047287142301
num neighbors 28, r2 score 0.40010534929374886
num neighbors 29, r2 score 0.4000884695549648
num neighbors 30, r2 score 0.3999541775672474
num neighbors 31, r2 score 0.3998452500971279
num neighbors 32, r2 score 0.3996481383068262
num neighbors 33, r2 score 0.3994516765324686
num neighbors 34, r2 score 0.39924114

In [9]:
best_k,max_r2_score # for label encoded 24 knn is best

(24, 0.40037727077854546)

In [ ]:
# for one hot encoded
y_train_np = y_train.to_numpy()
best_k = -100
max_r2_score = -100
for nn in range(1, 100):
    neigh_faissgpu = FaissKNeighbors_reg_gpu(k = nn)
    neigh_faissgpu.fit(X_train, y_train_np)
    y_test_predgpu_oh = neigh_faissgpu.predict(X_test)
    r2_score_now = r2_score(y_test, y_test_predgpu_oh)
    print(f'''num neighbors {nn}, r2 score {r2_score_now}''')
    if r2_score_now > max_r2_score:
        max_r2_score = r2_score_now
        best_k = nn

num neighbors 1, r2 score 0.04780358239749183
num neighbors 2, r2 score 0.28042038939095737
num neighbors 3, r2 score 0.35536173830996565
num neighbors 4, r2 score 0.393087606076013
num neighbors 5, r2 score 0.4146945702720689
num neighbors 6, r2 score 0.42899068236882965
num neighbors 7, r2 score 0.43870330543726765
num neighbors 8, r2 score 0.44571126592871413
num neighbors 9, r2 score 0.45111229354222737
num neighbors 10, r2 score 0.455232992849551
num neighbors 27, r2 score 0.47140483444071335
num neighbors 28, r2 score 0.47150606297383735
num neighbors 29, r2 score 0.471578304321823
num neighbors 30, r2 score 0.4715370381452074
num neighbors 31, r2 score 0.4715579209234946
num neighbors 32, r2 score 0.4715444566705641
num neighbors 33, r2 score 0.47148436054465614
num neighbors 34, r2 score 0.4714370509763057
num neighbors 35, r2 score 0.47131289499590223
num neighbors 36, r2 score 0.4711543354148542
num neighbors 37, r2 score 0.4712048924125418
num neighbors 38, r2 score 0.471203

In [8]:
print(best_k, max_r2_score) # for one hot 29 knn is best

29 0.471578304321823


In [ ]:
# next section

In [3]:
df_all = pd.read_pickle("./data_frames/dense_acs_mm_notoh.pkl")

In [4]:
cat_cols_sig = ['OCCP', 'SCHL', 'ST', 'JWTRNS', 'DRAT', 'COW', 'SEX', \
       'RELSHIPP', 'POBP', 'ENG', 'MAR', 'RAC1P'] # significant features from the earlier analysis
numeric_cols = ['WKHP', 'AGEP', 'PINCP']
print(len(cat_cols_sig), len(numeric_cols))

12 3


In [5]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1569188 entries, 0 to 1630166
Data columns (total 15 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   OCCP      1569188 non-null  int64  
 1   WKHP      1569188 non-null  float64
 2   AGEP      1569188 non-null  float64
 3   SCHL      1569188 non-null  int64  
 4   ST        1569188 non-null  int64  
 5   JWTRNS    1569188 non-null  int64  
 6   DRAT      1569188 non-null  int64  
 7   COW       1569188 non-null  int64  
 8   SEX       1569188 non-null  int64  
 9   RELSHIPP  1569188 non-null  int64  
 10  POBP      1569188 non-null  int64  
 11  ENG       1569188 non-null  int64  
 12  MAR       1569188 non-null  int64  
 13  RAC1P     1569188 non-null  int64  
 14  PINCP     1569188 non-null  float64
dtypes: float64(3), int64(12)
memory usage: 191.6 MB


In [13]:
df_all.head() #numeric has been mm scaled, categoricals still as they were in ACS encoded

,OCCP,WKHP,AGEP,SCHL,ST,JWTRNS,DRAT,COW,SEX,RELSHIPP,POBP,ENG,MAR,RAC1P,PINCP
0,299.0,0.295918,0.025316,17.0,0.0,0.0,0.0,1.0,1.0,18.0,47.0,0.0,4.0,0.0,0.039472
1,292.0,0.397959,0.037975,18.0,0.0,1.0,0.0,0.0,0.0,18.0,10.0,0.0,4.0,1.0,0.030967
2,325.0,0.173469,0.025316,17.0,0.0,0.0,0.0,1.0,1.0,18.0,18.0,0.0,4.0,0.0,0.030467
3,268.0,0.051020,0.215190,18.0,0.0,12.0,0.0,1.0,1.0,18.0,14.0,0.0,2.0,0.0,0.053479
4,161.0,0.091837,0.025316,17.0,0.0,0.0,0.0,0.0,0.0,18.0,32.0,0.0,4.0,0.0,0.009455


In [6]:
from utils import ordinal_encoder

In [7]:
df_all = ordinal_encoder(df_all, cat_cols_sig)

In [69]:
Counter(df_all['RAC1P'])

Counter({0.0: 1045807,
         8.0: 168468,
         1.0: 136227,
         5.0: 100852,
         7.0: 97582,
         2.0: 14455,
         6.0: 2755,
         4.0: 2316,
         3.0: 726})

In [14]:
from sklearn.model_selection import train_test_split
rand_seed = 21
df_train, df_test = train_test_split(df_all, test_size=0.2, random_state = rand_seed)

In [15]:
df_train.head()

,OCCP,WKHP,AGEP,SCHL,ST,JWTRNS,DRAT,COW,SEX,RELSHIPP,POBP,ENG,MAR,RAC1P,PINCP
1282566,34.0,0.071429,0.468354,19.0,39.0,11.0,0.0,0.0,1.0,5.0,32.0,1.0,4.0,1.0,0.059483
979236,269.0,0.397959,0.278481,3.0,31.0,1.0,0.0,0.0,1.0,1.0,55.0,0.0,0.0,0.0,0.139527
1495594,260.0,0.397959,0.316456,18.0,45.0,0.0,0.0,0.0,1.0,1.0,46.0,0.0,0.0,0.0,0.155536
1380416,268.0,0.397959,0.468354,18.0,42.0,1.0,0.0,0.0,0.0,5.0,43.0,0.0,2.0,0.0,0.029466
921721,323.0,0.397959,0.506329,15.0,29.0,1.0,0.0,2.0,1.0,1.0,30.0,0.0,0.0,0.0,0.159538


In [16]:
from collections import Counter
Counter(df_train['RAC1P'])

Counter({0.0: 836563,
         8.0: 134697,
         1.0: 108927,
         5.0: 80758,
         7.0: 78125,
         2.0: 11646,
         6.0: 2207,
         4.0: 1854,
         3.0: 573})

In [17]:
y_train = df_train['PINCP']
X_train = df_train.drop(['PINCP'], axis=1, inplace = False)

In [18]:
y_test = df_test['PINCP']
X_test = df_test.drop(['PINCP'], axis=1, inplace = False)

# Grid search for "best" decision tree

In [47]:
from sklearn.model_selection import KFold, GridSearchCV
# cv = KFold(n_splits=5)

In [121]:
grid = dict()
max_depth = np.arange(1, 20)
splitter = ['best', 'random']
min_samples_leaf = [0.001, 0.01, 0.1]
min_samples_split = [0.001, 0.005]


grid_dt = {'max_depth': max_depth,
               'splitter': splitter,
               'min_samples_leaf': min_samples_leaf,
               'min_samples_split': min_samples_split}

In [122]:
grid_dt

{'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19]),
 'splitter': ['best', 'random'],
 'min_samples_leaf': [0.001, 0.01, 0.1],
 'min_samples_split': [0.001, 0.005]}

In [ ]:
from sklearn import tree
dtree_reg = tree.DecisionTreeRegressor()
grid_search = GridSearchCV(estimator = dtree_reg, param_grid = grid_dt, n_jobs=4, verbose=10)
grid_result = grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 228 candidates, totalling 1140 fits


In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [76]:
from sklearn import tree
dtree_reg = tree.DecisionTreeRegressor()

In [77]:
grid_search = GridSearchCV(estimator=dtree_reg, param_grid=grid, n_jobs=4, cv=cv)

In [78]:
grid_result = grid_search.fit(X_train, y_train)

In [79]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Best: 0.493448 using {'max_depth': 13}


In [81]:
dtree_reg_best = tree.DecisionTreeRegressor(max_depth = 13)

In [82]:
dtree_reg_best.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=13)

In [84]:
dtree_reg_best.score(X_train, y_train)

0.5258775521586688

In [83]:
dtree_reg_best.score(X_test, y_test)

0.5002549062826196

# Grid search for "best" random forest

In [95]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 20, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid

In [101]:
n_estimators = [10, 100, 500]
max_depth = [5, 10, 15]

In [102]:
grid_rf = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [104]:
rf = RandomForestRegressor()
cv = KFold(n_splits=3)
grid_search = GridSearchCV(estimator=rf, param_grid = grid_rf, n_jobs=4, cv=cv)
grid_result = grid_search.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

{'n_estimators': [10, 100, 500],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [5, 10, 15],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [ ]:
grid_search = GridSearchCV(estimator=dtree_reg, param_grid=grid, n_jobs=4, cv=cv)

In [ ]:
rf_random.best_params_

In [107]:
rf8depth = RandomForestRegressor(n_estimators = 10, max_depth = 8)
rf8depth.fit(X_train,y_train)

RandomForestRegressor(max_depth=8, n_estimators=10)

In [108]:
rf.score(X_train, y_train), rf.score(X_test, y_test)

(0.4686889963825148, 0.47022364820004336)

In [109]:
rf5depth = RandomForestRegressor(n_estimators = 10, max_depth = 5)
rf5depth.fit(X_train,y_train)

RandomForestRegressor(max_depth=5, n_estimators=10)

In [110]:
rf5depth.score(X_train, y_train), rf5depth.score(X_test, y_test)

(0.40950996159653574, 0.4117925190246482)

In [111]:
rf10depth = RandomForestRegressor(n_estimators = 10, max_depth = 10)
rf10depth.fit(X_train,y_train)
rf10depth.score(X_train, y_train), rf10depth.score(X_test, y_test)

(0.49882551056576474, 0.4965951700895338)

In [115]:
max_depth = [5, 8, 10]
n_estimators = [10]

grid_rf = {'n_estimators': n_estimators,
               # 'max_features': max_features,
               'max_depth': max_depth,
               # 'min_samples_split': min_samples_split,
               # 'min_samples_leaf': min_samples_leaf,
               # 'bootstrap': bootstrap
          }
rf_temp = RandomForestRegressor()
grid_search = GridSearchCV(estimator=rf_temp, param_grid=grid_rf, cv = 2)
grid_result = grid_search.fit(X_train, y_train)

In [116]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.491849 using {'max_depth': 10, 'n_estimators': 10}


In [117]:
rf_5dep_100est = RandomForestRegressor(n_estimators = 100, max_depth = 5)
rf_5dep_100est.fit(X_train, y_train)

RandomForestRegressor(max_depth=5)

In [118]:
rf_5dep_100est.score(X_train, y_train)

0.4108138805594156

In [119]:
rf_5dep_100est.score(X_test, y_test)

0.41329075810028315

# Gradient boosted regressor

In [29]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
gbr_good = GradientBoostingRegressor(max_depth = 8, n_estimators=200, learning_rate = 0.1)
gbr_good.fit(X_train, y_train)

CPU times: user 13min 53s, sys: 67 ms, total: 13min 53s
Wall time: 13min 53s


GradientBoostingRegressor(max_depth=8, n_estimators=200)

In [30]:
gbr_good.score(X_train, y_train), gbr_good.score(X_test, y_test)

(0.5891244709897457, 0.5712725552712814)

# Knn regressor

In [19]:
from sklearn.neighbors import KNeighborsRegressor
neigh5 = KNeighborsRegressor(n_neighbors = 5)
neigh5.fit(X_train, y_train)

KNeighborsRegressor()

In [20]:
neigh5.score(X_train,y_train)

0.5670677879060622

In [21]:
neigh5.score(X_test,y_test)

0.34857123948720314

KNeighborsRegressor(n_neighbors=10)

In [23]:
neigh10.score(X_train,y_train), neigh10.score(X_test,y_test)

(0.49791175065745363, 0.38774123935632665)

# LightGBM 

In [1]:
import lightgbm as lgb

In [10]:
X_train

,OCCP,WKHP,AGEP,SCHL,ST,JWTRNS,DRAT,COW,SEX,RELSHIPP,POBP,ENG,MAR,RAC1P
1282566,34.0,0.071429,0.468354,19.0,39.0,11.0,0.0,0.0,1.0,5.0,32.0,1.0,4.0,1.0
979236,269.0,0.397959,0.278481,3.0,31.0,1.0,0.0,0.0,1.0,1.0,55.0,0.0,0.0,0.0
1495594,260.0,0.397959,0.316456,18.0,45.0,0.0,0.0,0.0,1.0,1.0,46.0,0.0,0.0,0.0
1380416,268.0,0.397959,0.468354,18.0,42.0,1.0,0.0,0.0,0.0,5.0,43.0,0.0,2.0,0.0
921721,323.0,0.397959,0.506329,15.0,29.0,1.0,0.0,2.0,1.0,1.0,30.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590612,45.0,0.377551,0.215190,19.0,14.0,1.0,0.0,0.0,1.0,1.0,15.0,0.0,0.0,0.0
65607,249.0,0.244898,0.696203,18.0,3.0,1.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
1176034,9.0,0.091837,0.708861,23.0,35.0,0.0,0.0,2.0,1.0,0.0,36.0,1.0,2.0,1.0
691694,148.0,0.397959,0.544304,20.0,19.0,1.0,0.0,2.0,1.0,1.0,38.0,0.0,0.0,0.0


In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression', # l2 loss
    'metric': ['l2'],
    'learning_rate': 0.05,
    # 'feature_fraction': 0.9,
    # 'bagging_fraction': 0.7,
    # 'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 12,
    # "num_leaves": 128,  
    # "max_bin": 512,
    "num_iterations": 1000
}
gbr_lgb = lgb.LGBMRegressor(**hyper_params)

gbr_lgb.fit(X_train, y_train, categorical_feature=cat_cols_sig)

In [21]:
gbr_lgb.score(X_train, y_train), gbr_lgb.score(X_test, y_test) # great on 200 estimators and 8 depth

(0.5883096237057377, 0.5729333048560534)

In [23]:
gbr_lgb.score(X_train, y_train), gbr_lgb.score(X_test, y_test) # great on 500 estimators and 8 depth

(0.6040130851315593, 0.5749567059056974)

In [25]:
gbr_lgb.score(X_train, y_train), gbr_lgb.score(X_test, y_test) # great on 500 estimators and 12 depth

(0.604980330700664, 0.5748615672982282)

In [ ]:
gbr_lgb.score(X_train, y_train), gbr_lgb.score(X_test, y_test) 
# 1000 estimators, 0.05 lr, 12 depth

# sklearn MLP regressor

## one hot encoded

In [1]:
from sklearn.neural_network import MLPRegressor

In [17]:
X_train.shape, X_test.shape

((1255350, 897), (313838, 897))

In [11]:
mlp_batch500 = MLPRegressor(hidden_layer_sizes = (500,),verbose=True, early_stopping=True)

In [ ]:
mlp_batch500.fit(X_train, y_train)

Iteration 1, loss = 0.00981532
Validation score: 0.554121
Iteration 2, loss = 0.00927378
Validation score: 0.563305
Iteration 3, loss = 0.00913620
Validation score: 0.566057
Iteration 4, loss = 0.00903703
Validation score: 0.566520
Iteration 5, loss = 0.00896938
Validation score: 0.568511
Iteration 6, loss = 0.00893522
Validation score: 0.566574
Iteration 7, loss = 0.00890637
Validation score: 0.567062
Iteration 8, loss = 0.00888224
Validation score: 0.564003
Iteration 9, loss = 0.00886382
Validation score: 0.567316
Iteration 10, loss = 0.00884524
Validation score: 0.567659
Iteration 11, loss = 0.00883015
Validation score: 0.566643
Iteration 12, loss = 0.00882105
Validation score: 0.565987
Iteration 13, loss = 0.00881129
Validation score: 0.564990
Iteration 14, loss = 0.00880029
Validation score: 0.565676


In [ ]:
import joblib
joblib.dump(mlp_batch500, "./onehot_encoded/models/mlp_sklearnbatch/sksingle_hidden500_mlp.pkl") 

In [ ]:
import joblib
joblib.dump(, "./onehot_encoded/models/mlp_reg/single_hidden500_mlp.pkl") 

In [15]:
mlp_batch500.score(X_test, y_test)

0.566416096870804

In [19]:
with open('./onehot_encoded/models/mlp_reg/single_hidden500_mlp.pkl', 'rb') as f:
    mlp_online = pickle.load(f)

In [20]:
y_pred_test = mlp_online.predict_many(X_test)

In [21]:
r2_score(y_test, y_pred_test)

-3.126959197402784

## label encoded

In [34]:
from collections import Counter
Counter(X_train_noh['RAC1P'])

Counter({0.0: 836563,
         8.0: 134697,
         1.0: 108927,
         5.0: 80758,
         7.0: 78125,
         2.0: 11646,
         6.0: 2207,
         4.0: 1854,
         3.0: 573})

In [36]:
X_train_noh.shape

(1255350, 14)

In [49]:
mlp_batch_le = MLPRegressor(hidden_layer_sizes = (14,),verbose=True, early_stopping=True, n_iter_no_change = 50)

In [50]:
mlp_batch_le.fit(X_train_noh, y_train_noh)

Iteration 1, loss = 2.40124932
Validation score: -0.472782
Iteration 2, loss = 0.02012458
Validation score: 0.244532
Iteration 3, loss = 0.01600113
Validation score: 0.337276
Iteration 4, loss = 0.01557154
Validation score: 0.342495
Iteration 5, loss = 0.01536744
Validation score: 0.142247
Iteration 6, loss = 0.01517268
Validation score: 0.358604
Iteration 7, loss = 0.01454504
Validation score: 0.392130
Iteration 8, loss = 0.01469092
Validation score: 0.309983
Iteration 9, loss = 0.01450414
Validation score: 0.232841
Iteration 10, loss = 0.01474247
Validation score: -0.255834
Iteration 11, loss = 0.01462555
Validation score: 0.103958
Iteration 12, loss = 0.01443994
Validation score: 0.399323
Iteration 13, loss = 0.01445763
Validation score: 0.401341
Iteration 14, loss = 0.01445696
Validation score: 0.000513
Iteration 15, loss = 0.01426221
Validation score: 0.300948
Iteration 16, loss = 0.01449980
Validation score: -0.090626
Iteration 17, loss = 0.01423983
Validation score: 0.387014
Ite

/home/jupyter-kriach/.conda/envs/biasbounties/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(early_stopping=True, hidden_layer_sizes=(14,), n_iter_no_change=50,
             verbose=True)

In [52]:
mlp_batch_le.score(X_test_noh, y_test_noh) # worse than linear, but its using label encoded data

0.44959839481182173

# target encoding

In [69]:
from sklearn.preprocessing import TargetEncoder
def target_encoder2(df, x_cols, y_col):
    df_new = df.copy()
    enc_auto = TargetEncoder(smooth="auto", target_type="continuous")
    df_new[x_cols] = enc_auto.fit_transform(df_new[x_cols], df[y_col])
    return df_new, enc_auto

In [70]:
df_notoh = pd.read_pickle("./data_frames/dense_acs_mm_notoh.pkl")

In [71]:
rand_seed = 21
df_train_te, df_test_te = train_test_split(df_notoh, test_size = 0.2, random_state = rand_seed)

In [60]:
# rand_seed = 21 
# X_train_te, X_test_te, y_train_te, y_test_te = train_test_split(df_notoh.drop('PINCP', axis=1), 
#                                                     df_notoh['PINCP'], test_size = 0.2,
#                                                     random_state = rand_seed)

In [62]:
cat_cols_sig = ['OCCP', 'SCHL', 'ST', 'JWTRNS', 'DRAT', 'COW', 'SEX', \
       'RELSHIPP', 'POBP', 'ENG', 'MAR', 'RAC1P'] 

In [72]:
df_train_te, enc_te = target_encoder2(df_train_te, cat_cols_sig, 'PINCP')

In [73]:
X_train_te = df_train_te.drop('PINCP', axis=1)
y_train_te = df_train_te['PINCP']

In [74]:
mlp_batch_te = MLPRegressor(hidden_layer_sizes = (14,),verbose=True, early_stopping=True, n_iter_no_change = 50)
mlp_batch_te.fit(X_train_te, y_train_te)

Iteration 1, loss = 0.01111675
Validation score: 0.490562
Iteration 2, loss = 0.01030261
Validation score: 0.501061
Iteration 3, loss = 0.01012518
Validation score: 0.507723
Iteration 4, loss = 0.01006232
Validation score: 0.512888
Iteration 5, loss = 0.00993577
Validation score: 0.518316
Iteration 6, loss = 0.00986621
Validation score: 0.516199
Iteration 7, loss = 0.00983115
Validation score: 0.516953
Iteration 8, loss = 0.00980796
Validation score: 0.504211
Iteration 9, loss = 0.00979512
Validation score: 0.523291
Iteration 10, loss = 0.00978710
Validation score: 0.520502
Iteration 11, loss = 0.00978080
Validation score: 0.524044
Iteration 12, loss = 0.00975722
Validation score: 0.521435
Iteration 13, loss = 0.00974367
Validation score: 0.520446
Iteration 14, loss = 0.00972694
Validation score: 0.526204
Iteration 15, loss = 0.00971163
Validation score: 0.523730
Iteration 16, loss = 0.00970005
Validation score: 0.524560
Iteration 17, loss = 0.00969205
Validation score: 0.523509
Iterat

/home/jupyter-kriach/.conda/envs/biasbounties/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor(early_stopping=True, hidden_layer_sizes=(14,), n_iter_no_change=50,
             verbose=True)

In [81]:
X_train_te.shape

(1255350, 14)

In [82]:
mlp_batch_te.score(X_train_te, y_train_te)

0.5384226336014757

In [78]:
type(X_test_te)

pandas.core.frame.DataFrame

In [77]:
mlp_batch_te.score(enc_te.transform(X_test_te), y_test_te)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- AGEP
- WKHP
